<a href="https://colab.research.google.com/github/Guhan2348519/demo/blob/main/2348519_NLP_Labtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
/content/drive/MyDrive/Tweets.csv

In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [5]:
data=pd.read_csv('/content/drive/MyDrive/Tweets.csv')
data.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,5.703060e+17,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,24-02-2015 11:35,NaN,Eastern Time (US & Canada)
1,5.703010e+17,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
2,5.703010e+17,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,24-02-2015 11:15,Lets Play,Central Time (US & Canada)
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,24-02-2015 11:15,NaN,Pacific Time (US & Canada)
4,5.703010e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,24-02-2015 11:14,NaN,Pacific Time (US & Canada)


In [46]:
actualSentiment = df['airline_sentiment']
positiveNegative = actualSentiment.map(lambda x: 1 if x=='positive' else 0)
df['Sentiment'] = positiveNegative
df['Sentiment'].value_counts()


Sentiment
0    9178
1    2363
Name: count, dtype: int64

In [47]:
# Preprocessing
# Assuming 'text' column contains the tweets and 'airline_sentiment' column contains sentiment labels
texts = df['text'].astype(str)
labels = df['airline_sentiment']
maxlen = 100  # Max length of each sequence
max_words = 10000  # Vocabulary size

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = pd.get_dummies(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)



Found 13233 unique tokens.


In [54]:
# RNN Model
model_rnn = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_words, 32),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes: negative, neutral, positive
])

model_rnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model_rnn.summary())

history_rnn = model_rnn.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)  # 20% of training data for validation



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 322179 (1.23 MB)
Trainable params: 322179 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
293/293 [==============================] - 9s 27ms/step - loss: 0.7847 - accuracy: 0.6689 - val_loss: 0.6531 - val_accuracy: 0.7140
Epoch 2/10
293/293 [==============================] - 9s 31ms/step - loss: 0.4867 - accuracy: 0.8193 - val_loss: 0.5810 - val_accuracy

In [55]:
# LSTM Model
model_lstm = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_words, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(3, activation='softmax')
])

model_lstm.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model_lstm.summary())

history_lstm = model_lstm.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)



Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 32)          320000    
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 328419 (1.25 MB)
Trainable params: 328419 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
293/293 [==============================] - 17s 50ms/step - loss: 0.7392 - accuracy: 0.6960 - val_loss: 0.6202 - val_accuracy: 0.7533
Epoch 2/10
293/293 [==============================] - 14s 49ms/step - loss: 0.4795 - accuracy: 0.8158 - val_loss: 0.5350 - val_accura

In [56]:
# Evaluate the models
loss_rnn, accuracy_rnn = model_rnn.evaluate(X_test, y_test)
print("RNN Test Accuracy:", accuracy_rnn)

loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test)
print("LSTM Test Accuracy:", accuracy_lstm)



92/92 [==============================] - 1s 6ms/step - loss: 1.0650 - accuracy: 0.7568
RNN Test Accuracy: 0.7568305730819702
92/92 [==============================] - 1s 11ms/step - loss: 1.0512 - accuracy: 0.7596
LSTM Test Accuracy: 0.7595628499984741


In [ ]:
import random

def predict_sentiment(text, model, tokenizer):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw, maxlen=200)
    prediction = int(model.predict(tw).round().item())
    return prediction

# Randomly select a text from the dataset
random_text = df['review'][random.randint(0, 5000)]

# Make predictions using the RNN and LSTM models
rnn_prediction = predict_sentiment(random_text, model_rnn, tokenizer)
lstm_prediction = predict_sentiment(random_text, model_lstm, tokenizer)

# Print the predictions
print("Random Text:")
print(random_text)
print("\nRNN Model Predicted label:", sentiment_label[1][rnn_prediction])
print("LSTM Model Predicted label:", sentiment_label[1][lstm_prediction])
